In [ ]:
from ultralytics import YOLO
import supervision as sv
import os
import cv2
import glob
import tqdm

from Modules.annotator import BoxAnnotator


videos_dir="Data/Raw_videos"
lines_model='Models/Lines_detection.pt'

In [ ]:
box_annotator = BoxAnnotator(
        thickness=2,
        text_thickness=1,
        text_scale=5
        )


def bounding_box_annotator(frame, filtered_zone_detections):
    
    # box annotator without filtering and tracking
    # labels = [f"{tracker_id} {model.names[class_id]}" for _,_,_, confidence, class_id, tracker_id in filtered_zone_detections]
    # labels = [f""]

    frame = box_annotator.annotate(scene=frame,
                                   detections=filtered_zone_detections,
                                   skip_label=False)
    return frame

def show_frames(frame):
    cv2.imshow('Road Lines', frame)
    if cv2.waitKey(5) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        

In [ ]:
# Load a model
model = YOLO(lines_model)  # pretrained YOLOv8n model

veh_names= model.names


def process_video(
    video_path='Example_Videos/new videos/new4.mp4',
    frames_dir_loc='FrameByFrame/original_Frames/',
    results_dir_loc = 'FrameByFrame/result_Frames/'
):
        
    frames_per_second = None

    
    video_name=video_path.split(os.sep)[-1]
    cap = cv2.VideoCapture(video_path)
    
    frames_per_second = cap.get(cv2.CAP_PROP_FPS)
    Total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    
    # Check if the video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        exit()

    # Create a directory to save the frames
    os.makedirs(frames_dir_loc, exist_ok=True)
    os.makedirs(results_dir_loc, exist_ok=True)
    
    # Read and save each frame
    frame_count = 0
    
    for i in tqdm.tqdm(range(int(Total_frames)),
                       desc=video_name ,
                       bar_format="Processing : {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]",ascii="□■□■█"):

        ret, frame = cap.read()
        if not ret:
            break  # Break the loop if the video is over

        frame_count += 1
        current_time_stamp = cap.get(cv2.CAP_PROP_POS_MSEC)
        
        # Run batched inference on all saved frames        
        results = model.track(frame,agnostic_nms=True,show=False,persist=True,verbose=False)[0]  # Assuming model can handle a list of file paths
        detections = sv.Detections.from_ultralytics(results)
        frame = bounding_box_annotator(frame, detections)
        
        show_frames(frame)
                
        # #turned off for now
        cv2.imwrite(f"{results_dir_loc}result_{frame_count:04d}.jpg", frame)
        if  frame_count > 2000 : break
        
    cap.release()
    yolo_model_names_list = model.names

In [ ]:
def frames2video(input_folder, output_video_path, fps=30):

    image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]
    image_files.sort() 
    img = cv2.imread(os.path.join(input_folder, image_files[0]))
    height, width, _ = img.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can change the codec based on your preference
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for image_file in image_files:
        img_path = os.path.join(input_folder, image_file)
        img = cv2.imread(img_path)
        video_writer.write(img)

    video_writer.release()    
    
    

In [ ]:
directory_name= videos_dir.split(os.sep)[-1]
videos_list = [i for i in glob.glob(videos_dir+os.sep+"*.mp4") ]

for each_video in videos_list:
            video_name = each_video.split(".mp4")[0].split(os.sep)[-1]
            results_dir_loc='Data/Detected_videos' + video_name+'/'
            process_video(video_path=each_video, results_dir_loc=results_dir_loc)
cv2.destroyAllWindows()

frames2video(results_dir_loc, 'Data/Detected_videos' + video_name +'.mp4')




In [ ]:
import cv2
import numpy as np
import glob,os

# Load the YOLOv8 model (replace with your specific setup)
from ultralytics import YOLO
lines_model='Models/Lines_detection.pt'
model = YOLO(lines_model)

# Load the image
images_src_dir='Data/Raw_images'
iamges_list= [i for i in glob.glob(images_src_dir+os.sep+"*.jpg") ]
output_dir='Data/Detected/images'

for imagefile in iamges_list:
    image = cv2.imread(imagefile)
    image_name=imagefile.split(os.sep)[-1]
    
    # Run inference
    results = model(image)

    # Iterate through detected bounding boxes
    for result in results[0].boxes.data:
        x_min, y_min, x_max, y_max, confidence, class_id = result.int().tolist()
        class_name = model.names[class_id]

        # Crop the ROI
        roi = image[y_min:y_max, x_min:x_max]

        # Convert ROI to HSV for color thresholding
        hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

        lower_white = np.array([0, 0, 200])  # Adjust as needed
        upper_white = np.array([180, 55, 255])

        # Define orange color range (tweak these values for road-line orange)
        lower_orange = np.array([15, 100, 150])  # Tighter bounds for saturation and brightness
        upper_orange = np.array([25, 255, 255])
        # Create masks for white and orange lines
        mask_white = cv2.inRange(hsv_roi, lower_white, upper_white)
        mask_orange = cv2.inRange(hsv_roi, lower_orange, upper_orange)

        # Create overlay colors
        light_green = np.array([0, 255, 0], dtype=np.uint8)  # Light green for white lines
        light_red = np.array([0, 0, 255], dtype=np.uint8)    # Light red for orange lines
        alpha = 0.5  # Transparency factor

        # Apply green overlay for white lines
        mask_white_3c = cv2.merge([mask_white, mask_white, mask_white])
        roi_with_white_overlay = np.where(mask_white_3c > 0,
                                        (alpha * light_green + (1 - alpha) * roi).astype(np.uint8),
                                        roi)

        # Apply red overlay for orange lines
        mask_orange_3c = cv2.merge([mask_orange, mask_orange, mask_orange])
        roi_with_overlay = np.where(mask_orange_3c > 0,
                                    (alpha * light_red + (1 - alpha) * roi_with_white_overlay).astype(np.uint8),
                                    roi_with_white_overlay)

        # Replace the modified ROI back into the original image
        image[y_min:y_max, x_min:x_max] = roi_with_overlay

        # Overlay the class name at the top of the line
        cv2.putText(image, class_name, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                    0.7, (0, 255, 0), 2, cv2.LINE_AA)

    # Save or display the resulting image
    cv2.imshow('Result', image)
    cv2.imwrite(output_dir+os.sep+image_name+'.jpg', image)
    cv2.destroyAllWindows()


In [3]:
import cv2
import numpy as np
import glob,os

# Load the YOLOv8 model (replace with your specific setup)
from ultralytics import YOLO
lines_model='Models/Lines_detection.pt'
model = YOLO(lines_model)

# Load the image
images_src_dir='Data/Raw_images'
iamges_list= [i for i in glob.glob(images_src_dir+os.sep+"*.jpg") ]
output_dir='Data/Detected/images'

for imagefile in iamges_list:
    image = cv2.imread(imagefile)
    image_name=imagefile.split(os.sep)[-1]
    
    # Run inference
    results = model(image)

    # Iterate through detected bounding boxes
    for result in results[0].boxes.data:

        x_min, y_min, x_max, y_max, confidence, class_id = result.int().tolist()
        class_name = model.names[class_id]

        # Crop the ROI
        roi = image[y_min:y_max, x_min:x_max]

        # Convert ROI to grayscale
        gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to reduce noise
        blurred_roi = cv2.GaussianBlur(gray_roi, (5, 5), 0)

        # Apply Canny edge detection
        edges = cv2.Canny(blurred_roi, threshold1=50, threshold2=150)

        # Find contours from edges
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Create a blank mask for contours
        contour_mask = np.zeros_like(roi)

        # Filter and draw contours
        for contour in contours:
            # Filter based on contour area or shape properties
            if cv2.contourArea(contour) > 100:  # Minimum area threshold
                cv2.drawContours(contour_mask, [contour], -1, (0, 255, 0), thickness=cv2.FILLED)  # Fill the contour

        # Apply transparency to the mask
        alpha = 0.5  # Transparency factor
        roi_with_overlay = cv2.addWeighted(contour_mask, alpha, roi, 1 - alpha, 0)

        # Replace the modified ROI back into the original image
        image[y_min:y_max, x_min:x_max] = roi_with_overlay

        # Overlay the class name at the top of the line
        cv2.putText(image, class_name, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                    0.7, (0, 255, 0), 2, cv2.LINE_AA)

    # Save or display the resulting image
    cv2.imshow('Result', image)
    cv2.imwrite(output_dir+os.sep+image_name+'.jpg', image)
    cv2.destroyAllWindows()



0: 384x640 (no detections), 50.4ms
Speed: 2.0ms preprocess, 50.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Strelka vlevo, 1 Strelka vpered, 50.0ms
Speed: 1.0ms preprocess, 50.0ms inference, 10.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Strelka vpered, 38.0ms
Speed: 3.6ms preprocess, 38.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Strelka vpered - vlevos, 10.0ms
Speed: 0.0ms preprocess, 10.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Liniya 1s, 1 Strelka vpered, 1 Strelka vpered - vpravo, 9.5ms
Speed: 0.0ms preprocess, 9.5ms inference, 5.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Liniya 1, 1 Perehod, 49.0ms
Speed: 0.0ms preprocess, 49.0ms inference, 5.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Strelka vpered, 12.7ms
Speed: 0.0ms preprocess, 12.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384